In [2]:
import platform

print(platform.python_version())

3.12.9


In [5]:
import os
import urllib.request

verdict_file = "the-verdict.txt"
if not os.path.exists(verdict_file):
    url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
    urllib.request.urlretrieve(url, verdict_file)

In [6]:
with open(verdict_file, "r", encoding="utf-8") as verdict:
    verdict_data = verdict.read()
    print(len(verdict_data))

20479


In [ ]:
import re

text = "The quick brown-- fox jumps over the lazy dog."
# result = re.split(r'(\s)', text)
re_exprs = r'([,.:;?_!"()\']|--|\s)'
result = re.split(re_exprs, text)
print(result)

['The', ' ', 'quick', ' ', 'brown', '--', '', ' ', 'fox', ' ', 'jumps', ' ', 'over', ' ', 'the', ' ', 'lazy', ' ', 'dog', '.', '']


In [14]:
filtered_result = re.split(re_exprs, verdict_data)
processed_data = [words.strip() for words in filtered_result if words.strip()]
print(len(processed_data))

4690


In [15]:
processed_data[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

In [16]:
sorted_data = sorted(set(processed_data))
print(len(sorted_data))

1130


Tokenizing

In [20]:
vocab = {token:index for index, token in enumerate(sorted_data)}
vocab['Jack']

57

In [21]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(re_exprs, text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [22]:
tokenizer = SimpleTokenizerV1(vocab)

sample_text = """It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""

In [23]:
ids = tokenizer.encode(sample_text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [24]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

Byte pair encoding

In [26]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


Data sampling with sliding window

In [32]:
enc_text = tokenizer.encode(verdict_data)
len(enc_text)

5145

In [33]:
enc_sample = enc_text[50:]
len(enc_sample)

5095

In [34]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [38]:
from torch.utils.data import Dataset
import torch


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [39]:
from torch.utils.data import DataLoader

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader